# N-body gravitational problem

## Fromal mathematical description
$$ \frac{d\textbf{r}_i}{dt} = \textbf{v}_i $$
$$ \frac{d\textbf{v}_i}{dt} = -\sum_{j \neq i}^N G \frac{(\textbf{r}_i-\textbf{r}_j)}{{|\textbf{r}_i-\textbf{r}_j|}^3}m_j $$

## Let us create a new module (for learning purposes)

In [1]:
module NBodyGravitational

using DiffEqBase, DifferentialEquations, StaticArrays, Plots, StaticArrays


grav_const = 6.673e-11

struct MassBody{mType, cType}
    m :: mType
    r :: SVector{3, cType}
    v :: SVector{3, cType}
    
    function MassBody{mType,cType}(m::mType, r::SVector{3, cType}, v::SVector{3, cType})
        @assert length(r)==length(v) "The length of the position (r) and velocity (v) components should match"
        new(m,r,v)
    end
end

MassBody(m::mType, r :: SVector{3, cType}, v :: SVector{3, cType}) where {mType <: AbstractFloat, cType  <: AbstractFloat} = MassBody{mType, cType}(m,r,v)

struct NBodyGravProblem
    bodies :: AbstractArray{MassBody, 1}
    tSpan :: Tuple{Float64, Float64}
    G :: Float64
end

NBodyGravProblem(bodies, tSpan::Tuple{Float64, Float64}) = NBodyGravProblem(bodies, tSpan, grav_const)

import Base.convert
convert(::Type{ODEProblem}, x:: NBodyGravProblem) =
begin
    n = length(x.bodies)
    u0 = zeros(6*n);
    m = zeros(n)
    
    for i=1:n
        ith = 3(i-1)+1;        
        u0[ith:ith+2] = x.bodies[i].r;
        u0[3n+ith:3n+ith+2] = x.bodies[i].v 
        m[i] = x.bodies[i].m
    end
    
    function grav_acceleration(m2, r1, r2)
        return -x.G*m2*(r1-r2)/norm(r1-r2)^3;
    end
    
    
    function ode_system!(du, u, p, t)
        for i=1:n
            ith = 3(i-1)+1;        
            du[ith:ith+2] = u[3n+ith:3n+ith+2];
            du[3n+ith:3n+ith+2] = 
            begin
                accel=zeros(3); 
                j=1; 
                while j<i 
                    jth = 3(j-1)+1;
                    accel += grav_acceleration(m[j],u[ith:ith+2],u[jth:jth+2])
                    j+=1
                end
                j+=1;
                while j<=n 
                    jth = 3(j-1)+1;
                    accel += grav_acceleration(m[j],u[ith:ith+2],u[jth:jth+2])
                    j+=1
                end
                accel
            end
        end 
    end

    ODEProblem(ode_system!, u0, x.tSpan)
end

import DiffEqBase.solve
solve(x::NBodyGravProblem) = 
begin
    return DiffEqBase.solve(convert(ODEProblem,x), alg_hints=[:stiff])
end

function plot_xy_scattering(solution::ODESolution, path::AbstractString, duration::AbstractFloat = 3.0)
    fps = 15
    n = Int(length(solution[1])/6)
    
    pl = plot()
    for i=1:n
        plot!(pl, solution, vars = (3(i-1)+1,3(i-1)+2), linecolor = :black, label = "Orbit $i")
    end
    
    tmax = maximum(solution.t);
    tstep = tmax/(fps*duration);
    animation_data = solution(0.0:tstep:tmax)
    pos0 = animation_data[1];
    
     scatter!([pos0[3(i-1)+1] for i=1:n]', [pos0[3(i-1)+2] for i=1:n]', markersize = 10, label = (["$i" for i=1:n]))
    
    anim = @animate for i=1:length(animation_data)
        for j=1:n
           pl[j+n] = animation_data[i][3(j-1)+1], animation_data[i][3(j-1)+2]
        end
    end
    gif(anim, path, fps = fps) 
end


function plot_xy_trailing(solution::ODESolution, path::AbstractString; ntrail::Int = 3, duration::AbstractFloat = 3.0)
    # ntrail - number of points for displaying a trailing path; the trail will correspond to the velocity of a body
    fps = 15
    n = Int(length(solution[1])/6)
    tmax = maximum(solution.t);
    tstep = tmax/(fps*duration);
    nshift = Int(floor(ntrail/2));
    animation_data = solution(tstep : tstep : tmax-tstep)
    xy_data = zeros(length(animation_data), 2n)
    
    for i=1:n
        xy_data[:,i] = [d[3(i-1)+1] for d in  animation_data]
        xy_data[:,i+n] = [d[3(i-1)+2] for d in  animation_data]
    end
    
    xlim = 1.1*[minimum(minimum(xy_data[:, 1:n])), maximum(maximum(xy_data[:, 1:n]))]
    ylim = 1.1*[minimum(minimum(xy_data[:, n+1:2n])), maximum(maximum(xy_data[:, n+1:2n]))]
    anim = @animate for i=1+nshift:length(animation_data)-nshift
        plot(xy_data[i-nshift:i+nshift,1:n],xy_data[i-nshift:i+nshift,n+1:2n], xlim = xlim, ylim = ylim, lw=5)
    end
    gif(anim, path, fps = 15)
end

export NBodyGravProblem, MassBody, plot_xy_trailing, plot_xy_scattering

end

NBodyGravitational

### Now, let us test the implementation of the n-body gravitational task on a simple example including Sun, Earth and Moon

In [2]:
using NBodyGravitational, DifferentialEquations, Plots, BenchmarkTools, StaticArrays
Sun = MassBody(1.989e30, SVector(0.0,0.0,0.0), SVector(0.0,0.0,0.0))
Earth = MassBody(5.972e24, SVector(149.6e9,0,0), SVector(0,-30e3,0))
Moon = MassBody(7.346e22, SVector(149.6e9 + 406e6,0,0), SVector(0,-30e3-1.022e3,0))
tspan = (0.0, 3600*24*365.0);
problem = NBodyGravProblem([Earth,Moon,Sun], tspan)
solution = solve(problem);
pl = plot(solution, vars = (4,5), label="Moon")
plot!(pl,solution, vars = (1,2), label="Earth")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -150000000000 
 
 
 -100000000000 
 
 
 -50000000000 
 
 
 0 
 
 
 50000000000 
 
 
 100000000000 
 
 
 150000000000 
 
 
 -150000000000 
 
 
 -100000000000 
 
 
 -50000000000 
 
 
 0 
 
 
 50000000000 
 
 
 100000000000 
 
 
 150000000000 
 
<polyline clip-path="url(#clip5102)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 559.531,194.076 559.525,195.248 559.507,196.42 559.477,197.591 559.436,198.761 559.383,199.93 559.319,201.098 559.243,202.264 559.156,203.429 559.058,204.591 
 558.949,205.751 558.829,206.909 558.698,208.064 558.558,209.216 558.407,210.366 558.246,211.512 558.076,212.656 557.896,213.796 557.707,214.932 557.509,216.065 
 557.302,217.195 557.087,218.322 556.863,219.444 556.632,220.563 556.392,221.679 556.145,222.791 555.891,223.899 555.629,225.004 555.36,226.105 555.084,227.203 
 554.802,228.298 554.513,229.389 554.218,230.477 553.917,231.561 553.61,232.643 553.296,233.721 552.977,234.796 552.653,235.869 552.323,236.939 551.987,238.006 
 551.646,239.071 551.3,240.133 550.948,241.193 550.592,242.251 550.23,243.308 549.863,244.362 549.491,245.415 549.113,246.466 548.731,247.515 548.343,248.563 
 547.95,249.609 547.551,250.655 547.147,251.699 546.738,252.743 546.323,253.787 545.902,254.829 545.475,255.871 545.042,256.912 544.604,257.953 544.159,258.993 
 543.707,260.033 543.249,261.073 542.783,262.113 542.311,263.152 541.832,264.191 541.345,265.23 540.851,266.268 540.349,267.306 539.84,268.344 539.322,269.381 
 538.796,270.418 538.261,271.455 537.718,272.491 537.166,273.526 536.604,274.56 536.034,275.593 535.455,276.626 534.866,277.656 534.268,278.686 533.66,279.714 
 533.041,280.74 532.413,281.764 531.774,282.786 531.124,283.806 530.465,284.823 529.795,285.837 529.115,286.849 528.426,287.857 527.725,288.862 527.015,289.863 
 526.295,290.861 525.564,291.854 524.822,292.842 524.071,293.825 523.31,294.803 522.539,295.776 521.758,296.743 520.967,297.705 520.168,298.662 519.358,299.613 
 518.54,300.558 517.714,301.497 516.879,302.429 516.037,303.354 515.187,304.273 514.33,305.185 513.465,306.09 512.592,306.988 511.713,307.88 510.826,308.764 
 509.933,309.642 509.034,310.513 508.13,311.377 507.22,312.234 506.304,313.084 505.384,313.928 504.458,314.765 503.526,315.594 502.59,316.418 501.65,317.234 
 500.704,318.043 499.755,318.847 498.802,319.644 497.844,320.434 496.883,321.219 495.918,321.997 494.95,322.769 493.978,323.535 493.003,324.295 492.024,325.049 
 491.042,325.798 490.056,326.54 489.068,327.277 488.077,328.008 487.083,328.735 486.086,329.456 485.086,330.172 484.084,330.883 483.079,331.59 482.07,332.291 
 481.059,332.988 480.046,333.68 479.029,334.367 478.009,335.05 476.986,335.728 475.961,336.402 474.933,337.073 473.901,337.74 472.866,338.403 471.828,339.063 
 470.787,339.719 469.742,340.371 468.693,341.02 467.641,341.665 466.585,342.306 465.525,342.944 464.461,343.578 463.393,344.208 462.32,344.835 461.244,345.458 
 460.161,346.078 459.074,346.695 457.981,347.308 456.883,347.917 455.779,348.523 454.67,349.125 453.556,349.723 452.436,350.317 451.311,350.907 450.18,351.492 
 449.043,352.074 447.899,352.651 446.749,353.222 445.593,353.789 444.43,354.35 443.261,354.907 442.086,355.458 440.904,356.003 439.716,356.543 438.522,357.078 
 437.322,357.607 436.115,358.13 434.903,358.647 433.684,359.156 432.459,359.659 431.229,360.155 429.992,360.644 428.75,361.126 427.502,361.6 426.249,362.068 
 424.99,362.528 423.726,362.98 422.456,363.425 421.182,363.863 419.902,364.292 418.619,364.713 417.332,365.126 416.04,365.53 414.745,365.926 413.446,366.314 
 412.143,366.693 410.836,367.064 409.526,367.426 408.213,367.78 406.897,368.125 405.578,368.462 404.255,368.79 402.931,369.109 401.606,369.42 400.278,369.722 
 398.95,370.015 397.62,370.3 396.288,370.576 394.955,370.844 393.622,371.103 392.287,371.354 390.951,371.596 389.615,371.83 388.279,372.055 386.942,3

### Now, let us simulate movement of Pluto and Charon

In [3]:
Pluto = MassBody(1.309e22, SVector(0.0,2.035e6,0.0), SVector(23.675,0.0,0.0))
Charon = MassBody(1.547e21, SVector(0.0,-1.7497e7,0.0), SVector(-200.374,0,0))
tspan = (0.0, 2500000.0);
problem = NBodyGravProblem([Pluto,Charon], tspan)
pl_ch_solution = solve(problem);
pl = plot(pl_ch_solution, vars = (1,2), label = "Pluto" )
plot!(pl,pl_ch_solution, vars = (4,5), label = "Charon" )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15000000 
 
 
 -10000000 
 
 
 -5000000 
 
 
 0 
 
 
 5000000 
 
 
 10000000 
 
 
 15000000 
 
 
 -15000000 
 
 
 -10000000 
 
 
 -5000000 
 
 
 0 
 
 
 5000000 
 
 
 10000000 
 
 
 15000000 
 
<polyline clip-path="url(#clip5302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 328.834,173.19 329.57,173.198 330.305,173.221 331.04,173.261 331.772,173.317 332.503,173.388 333.231,173.475 333.956,173.578 334.676,173.697 335.393,173.831 
 336.104,173.98 336.81,174.145 337.511,174.325 338.204,174.52 338.891,174.73 339.571,174.955 340.242,175.194 340.905,175.447 341.559,175.715 342.203,175.997 
 342.837,176.293 343.461,176.602 344.075,176.925 344.676,177.261 345.266,177.609 345.844,177.97 346.41,178.344 346.962,178.729 347.5,179.127 348.025,179.535 
 348.536,179.955 349.032,180.386 349.513,180.827 349.979,181.278 350.429,181.74 350.863,182.21 351.28,182.69 351.681,183.179 352.065,183.676 352.432,184.181 
 352.782,184.694 353.114,185.214 353.428,185.742 353.723,186.275 354,186.815 354.259,187.36 354.498,187.911 354.719,188.467 354.921,189.027 355.103,189.591 
 355.267,190.159 355.41,190.73 355.534,191.304 355.638,191.881 355.723,192.459 355.787,193.039 355.832,193.62 355.857,194.202 355.862,194.784 355.847,195.366 
 355.813,195.947 355.758,196.528 355.684,197.106 355.589,197.683 355.475,198.258 355.342,198.83 355.189,199.398 355.017,199.964 354.826,200.525 354.615,201.082 
 354.386,201.634 354.138,202.181 353.871,202.723 353.586,203.258 353.282,203.787 352.961,204.309 352.622,204.824 352.266,205.332 351.893,205.832 351.503,206.323 
 351.096,206.806 350.673,207.28 350.234,207.745 349.779,208.2 349.309,208.645 348.824,209.079 348.324,209.503 347.81,209.917 347.282,210.319 346.741,210.709 
 346.187,211.089 345.62,211.455 345.041,211.81 344.45,212.151 343.847,212.48 343.234,212.796 342.61,213.098 341.976,213.387 341.333,213.663 340.68,213.925 
 340.018,214.172 339.349,214.406 338.672,214.624 337.987,214.828 337.297,215.017 336.6,215.192 335.897,215.351 335.189,215.495 334.476,215.624 333.76,215.738 
 333.039,215.836 332.316,215.919 331.589,215.987 330.861,216.039 330.132,216.075 329.402,216.095 328.671,216.099 327.941,216.088 327.211,216.062 326.482,216.02 
 325.755,215.962 325.03,215.889 324.307,215.8 323.587,215.696 322.872,215.576 322.161,215.441 321.456,215.291 320.755,215.125 320.061,214.945 319.372,214.75 
 318.691,214.54 318.017,214.316 317.35,214.077 316.692,213.824 316.042,213.557 315.402,213.277 314.772,212.982 314.153,212.674 313.544,212.353 312.946,212.019 
 312.36,211.672 311.786,211.313 311.224,210.942 310.674,210.558 310.138,210.163 309.615,209.757 309.106,209.339 308.612,208.911 308.133,208.472 307.669,208.023 
 307.221,207.564 306.788,207.096 306.372,206.619 305.971,206.133 305.588,205.638 305.221,205.135 304.871,204.624 304.538,204.106 304.224,203.582 303.928,203.05 
 303.65,202.512 303.391,201.969 303.15,201.42 302.928,200.866 302.725,200.308 302.541,199.745 302.376,199.178 302.231,198.608 302.104,198.035 301.998,197.459 
 301.911,196.881 301.844,196.302 301.798,195.722 301.771,195.14 301.764,194.559 301.777,193.977 301.809,193.395 301.862,192.815 301.934,192.235 302.025,191.657 
 302.137,191.082 302.268,190.508 302.419,189.938 302.59,189.372 302.779,188.809 302.989,188.251 303.217,187.697 303.464,187.149 303.729,186.606 304.013,186.068 
 304.316,185.537 304.636,185.012 304.974,184.494 305.33,183.984 305.704,183.482 306.094,182.989 306.502,182.504 306.925,182.027 307.365,181.56 307.821,181.103 
 308.292,180.655 308.779,180.218 309.28,179.791 309.796,179.375 310.326,178.97 310.87,178.577 311.427,178.196 311.997,177.828 312.579,177.471 313.173,177.128 
 313.779,176.797 314.396,176.48 315.024,176.175 315.662,175.884 316.31,175.607 316.967,175.344 317.633,175.096 318.307,174.862 318.989,174.643 319.678,174.439 
 320.374,174.249 321.076,174.075 321.784,173.91

In [4]:
plot_xy_scattering(pl_ch_solution,"./anim_pluto_charon_scattering.gif", 4.0)

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_pluto_charon_scattering.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_pluto_charon_scattering.gif")

### Another interesting example consists of two bodies with equal masses moving on the same orbit: their orbits should coincide

The time for evolving the solution is less that the orbital period.

In [5]:
body1 = MassBody(2.0, SVector(0.0,1.0,0.0), SVector(5.775e-6,0.0,0.0))
body2 = MassBody(2.0, SVector(0.0,-1.0,0.0), SVector(-5.775e-6,0.0,0.0))
tspan = (0.0, 1111150.0);
problem = NBodyGravProblem([body1,body2], tspan)
solution = solve(problem);
pl = plot(solution, vars = (1,2))
plot!(pl,solution, vars = (4,5),xlim=(-1,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip1602)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 309.908,12.0811 311.645,12.0849 313.382,12.0961 315.119,12.1149 316.855,12.1411 318.592,12.1749 320.327,12.2162 322.063,12.2649 323.798,12.3212 325.532,12.3849 
 327.266,12.4561 328.999,12.5348 330.731,12.621 332.462,12.7147 334.192,12.8159 335.922,12.9245 337.65,13.0406 339.377,13.1642 341.103,13.2952 342.828,13.4337 
 344.551,13.5796 346.273,13.733 347.993,13.8938 349.712,14.062 351.429,14.2376 353.145,14.4207 354.858,14.6111 356.57,14.809 358.28,15.0142 359.988,15.2268 
 361.694,15.4468 363.397,15.6741 365.099,15.9088 366.798,16.1509 368.495,16.4003 370.189,16.6571 371.881,16.9212 373.57,17.1925 375.257,17.4712 376.941,17.7571 
 378.622,18.0503 380.301,18.3508 381.976,18.6585 383.649,18.9734 385.318,19.2955 386.985,19.6248 388.648,19.9613 390.308,20.305 391.965,20.6558 393.618,21.0138 
 395.268,21.3789 396.914,21.7511 398.557,22.1305 400.196,22.5169 401.831,22.9104 403.463,23.311 405.09,23.7187 406.714,24.1334 408.334,24.5551 409.949,24.9838 
 411.561,25.4195 413.168,25.8621 414.771,26.3116 416.369,26.7681 417.964,27.2315 419.553,27.7017 421.139,28.1788 422.719,28.6627 424.295,29.1534 425.867,29.6509 
 427.433,30.1551 428.995,30.6662 430.552,31.1839 432.104,31.7084 433.651,32.2395 435.193,32.7773 436.729,33.3217 438.261,33.8728 439.787,34.4305 441.308,34.9948 
 442.823,35.5656 444.333,36.1429 445.837,36.7268 447.336,37.3172 448.829,37.9141 450.316,38.5175 451.797,39.1272 453.272,39.7434 454.741,40.3659 456.205,40.9948 
 457.662,41.63 459.113,42.2714 460.559,42.9191 461.997,43.5731 463.43,44.2332 464.856,44.8995 466.276,45.572 467.69,46.2505 469.097,46.9352 470.497,47.6259 
 471.891,48.3226 473.278,49.0253 474.658,49.734 476.032,50.4486 477.399,51.1692 478.759,51.8956 480.112,52.6279 481.458,53.366 482.797,54.1099 484.129,54.8596 
 485.454,55.6151 486.771,56.3762 488.082,57.143 489.385,57.9155 490.68,58.6936 491.968,59.4772 493.248,60.2664 494.521,61.0612 495.786,61.8614 497.043,62.6671 
 498.292,63.4782 499.534,64.2947 500.768,65.1166 501.993,65.9438 503.211,66.7763 504.421,67.614 505.623,68.4569 506.816,69.305 508.002,70.1583 509.179,71.0166 
 510.348,71.88 511.509,72.7485 512.661,73.6219 513.806,74.5003 514.941,75.3836 516.069,76.2718 517.188,77.1648 518.298,78.0627 519.4,78.9653 520.493,79.8727 
 521.578,80.7848 522.654,81.7015 523.721,82.6229 524.779,83.5488 525.829,84.4794 526.87,85.4144 527.901,86.354 528.924,87.298 529.938,88.2464 530.943,89.1991 
 531.939,90.1563 532.926,91.1177 533.903,92.0834 534.872,93.0533 535.831,94.0275 536.781,95.0057 537.721,95.9878 538.651,96.974 539.572,97.9642 540.483,98.9584 
 541.385,99.9564 542.277,100.958 543.159,101.964 544.032,102.974 544.895,103.987 545.748,105.004 546.591,106.025 547.424,107.049 548.248,108.077 549.062,109.108 
 549.866,110.143 550.66,111.181 551.444,112.223 552.218,113.268 552.982,114.317 553.736,115.368 554.48,116.423 555.214,117.482 555.938,118.543 556.652,119.607 
 557.355,120.675 558.049,121.745 558.732,122.819 559.405,123.895 560.068,124.975 560.721,126.057 561.363,127.142 561.995,128.23 562.617,129.32 563.228,130.414 
 563.829,131.509 564.42,132.608 565,133.709 565.569,134.813 566.128,135.919 566.677,137.027 567.215,138.138 567.743,139.251 568.26,140.367 568.766,141.485 
 569.262,142.605 569.747,143.727 570.221,144.851 570.683,145.977 571.135,147.104 571.576,148.234 572.006,149.366 572.425,150.499 572.833,151.634 573.23,152.771 
 573.616,153.909 573.992,155.049 574.356,156.191 574.709,157.334 575.051,158.479 575.383,159.625 575.703,160.772 576.012,161.921 576.311,163.071 576.598,164.223 
 576.874,165.376 577.14,166.529 577.394,167.684 577.637,168.841 577.87,169.998 578.091,171.156 578.301,172.315 578.5,173.475 578.688,174.636

In [6]:
plot_xy_trailing(solution, "./anim_two_boddies_trailing.gif")

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_two_boddies_trailing.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_two_boddies_trailing.gif")

In [7]:
plot_xy_scattering(solution,"./anim_two_boddies_scattering.gif")

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_two_boddies_scattering.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_two_boddies_scattering.gif")

If we add one more body with the same mass as two others, and place the third body to the baricenter of the previous system, the picture will be different.

In [8]:
body3 = MassBody(2.0, SVector(0.0,0.0,0.0), SVector(0.0,0.0,0.0))
tspan = (0.0, 200000.0);
problem = NBodyGravProblem([body1, body2, body3], tspan)
solution = solve(problem);
pl = plot(solution, vars = (1,2), label="body1")
plot!(pl,solution, vars = (4,5), label="body2")
plot!(pl,solution, vars = (7,8), label="body3", marker=:circle, ylim=(-1.1, 1.1), xlim=(-1.1, 1.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip0602)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 309.908,28.3591 310.193,28.3597 310.477,28.3613 310.761,28.3641 311.045,28.368 311.33,28.3729 311.614,28.379 311.898,28.3862 312.182,28.3945 312.466,28.4039 
 312.75,28.4144 313.034,28.426 313.318,28.4388 313.603,28.4526 313.887,28.4675 314.171,28.4836 314.455,28.5008 314.739,28.519 315.022,28.5384 315.306,28.5589 
 315.59,28.5805 315.874,28.6032 316.158,28.627 316.441,28.6519 316.725,28.6779 317.009,28.705 317.292,28.7333 317.576,28.7626 317.859,28.7931 318.143,28.8247 
 318.426,28.8574 318.71,28.8912 318.993,28.9261 319.276,28.9621 319.559,28.9992 319.842,29.0375 320.125,29.0768 320.408,29.1173 320.691,29.1589 320.974,29.2016 
 321.257,29.2454 321.539,29.2903 321.822,29.3364 322.104,29.3835 322.387,29.4318 322.669,29.4812 322.951,29.5317 323.233,29.5833 323.515,29.636 323.797,29.6899 
 324.079,29.7449 324.361,29.801 324.643,29.8582 324.924,29.9166 325.206,29.976 325.487,30.0366 325.768,30.0983 326.049,30.1611 326.331,30.2251 326.611,30.2902 
 326.892,30.3564 327.173,30.4237 327.454,30.4921 327.734,30.5617 328.014,30.6324 328.295,30.7043 328.575,30.7772 328.855,30.8513 329.134,30.9266 329.414,31.0029 
 329.694,31.0804 329.973,31.159 330.252,31.2388 330.531,31.3197 330.81,31.4017 331.089,31.4849 331.368,31.5692 331.646,31.6546 331.925,31.7412 332.203,31.8289 
 332.481,31.9177 332.759,32.0077 333.037,32.0989 333.314,32.1911 333.592,32.2845 333.869,32.3791 334.146,32.4748 334.423,32.5716 334.7,32.6696 334.976,32.7688 
 335.253,32.8691 335.529,32.9705 335.805,33.0731 336.081,33.1768 336.356,33.2817 336.632,33.3877 336.907,33.4949 337.182,33.6033 337.457,33.7128 337.732,33.8235 
 338.006,33.9353 338.28,34.0483 338.554,34.1625 338.828,34.2778 339.102,34.3943 339.375,34.5119 339.649,34.6308 339.922,34.7507 340.194,34.8719 340.467,34.9942 
 340.739,35.1177 341.012,35.2424 341.283,35.3682 341.555,35.4952 341.827,35.6234 342.098,35.7528 342.369,35.8833 342.639,36.0151 342.91,36.148 343.18,36.2821 
 343.45,36.4174 343.72,36.5538 343.989,36.6915 344.258,36.8303 344.527,36.9704 344.796,37.1116 345.064,37.254 345.332,37.3977 345.6,37.5425 345.868,37.6885 
 346.135,37.8357 346.402,37.9842 346.669,38.1338 346.935,38.2846 347.202,38.4366 347.467,38.5899 347.733,38.7443 347.998,38.9 348.263,39.0569 348.528,39.2149 
 348.792,39.3742 349.056,39.5348 349.32,39.6965 349.583,39.8594 349.847,40.0236 350.109,40.189 350.372,40.3556 350.634,40.5234 350.896,40.6925 351.157,40.8628 
 351.418,41.0343 351.679,41.207 351.94,41.381 352.2,41.5562 352.46,41.7326 352.719,41.9103 352.978,42.0892 353.237,42.2695 353.495,42.451 353.753,42.6337 
 354.011,42.8177 354.268,43.0029 354.525,43.1894 354.782,43.3771 355.038,43.5661 355.294,43.7564 355.55,43.9479 355.805,44.1407 356.059,44.3347 356.314,44.53 
 356.568,44.7266 356.821,44.9245 357.075,45.1236 357.327,45.3241 357.58,45.5258 357.832,45.7288 358.083,45.9331 358.334,46.1386 358.585,46.3455 358.835,46.5537 
 359.085,46.7632 359.335,46.9739 359.584,47.186 359.832,47.3994 360.08,47.6141 360.328,47.8301 360.575,48.0474 360.822,48.2661 361.068,48.4861 361.314,48.7073 
 361.559,48.93 361.804,49.1539 362.048,49.3792 362.292,49.6058 362.535,49.8338 362.778,50.0631 363.021,50.2937 363.263,50.5257 363.504,50.759 363.745,50.9937 
 363.985,51.2298 364.225,51.4672 364.464,51.7059 364.703,51.9461 364.941,52.1875 365.179,52.4304 365.416,52.6746 365.652,52.9202 365.888,53.1672 366.124,53.4155 
 366.359,53.6653 366.593,53.9164 366.827,54.1689 367.06,54.4228 367.292,54.6781 367.524,54.9348 367.756,55.1928 367.987,55.4523 368.217,55.7132 368.446,55.9755 
 368.676,56.2392 368.904,56.5043 369.132,56.7708 369.359,57.0387 369.585,57.3081 369.811,57.5789 370.037,57.8511 370.261,58.1247 370.485,58.4 370.7

## Choreography
### The Figure 8

In [9]:
G=1
b1 = MassBody(1.0, SVector(-0.97000436, 0.24308753, 0.0), SVector(-0.5*0.93240737, -0.5*0.86473146, 0.0))
b2 = MassBody(1.0, SVector(0.97000436, -0.24308753, 0.0), SVector(-0.5*0.93240737, -0.5*0.86473146, 0.0))
b3 = MassBody(1.0, SVector(0.0, 0.0, 0.0), SVector(0.93240737, 0.86473146, 0.0))
tspan = (0.0, 6.322); # period is about 6.322 seconds
problem = NBodyGravProblem([b1, b2, b3], tspan, G)
solution8 = solve(problem);
pl = plot(solution8, vars = (1,2))
plot!(pl,solution8, vars = (4,5))
plot!(pl,solution8, vars = (7,8), xlim=(-1.2,1.2))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -0.3 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
<polyline clip-path="url(#clip0802)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 91.3289,70.297 90.6696,71.6906 90.0211,73.0902 89.3833,74.4953 88.7562,75.906 88.1397,77.3218 87.5336,78.7425 86.938,80.168 86.3526,81.5981 85.7774,83.0324 
 85.2124,84.4709 84.6573,85.9133 84.1122,87.3594 83.5769,88.8091 83.0513,90.2622 82.5354,91.7185 82.0291,93.1778 81.5322,94.64 81.0448,96.105 80.5666,97.5727 
 80.0977,99.0427 79.6379,100.515 79.1872,101.99 78.7455,103.466 78.3126,104.945 77.8886,106.426 77.4733,107.908 77.0666,109.392 76.6686,110.877 76.279,112.364 
 75.8979,113.852 75.5252,115.342 75.1608,116.833 74.8045,118.324 74.4565,119.817 74.1165,121.311 73.7845,122.806 73.4605,124.301 73.1444,125.798 72.8361,127.295 
 72.5356,128.792 72.2428,130.291 71.9576,131.789 71.68,133.289 71.41,134.788 71.1474,136.289 70.8923,137.789 70.6445,139.29 70.4041,140.791 70.1709,142.293 
 69.945,143.795 69.7263,145.297 69.5147,146.799 69.3102,148.301 69.1127,149.803 68.9223,151.306 68.7388,152.808 68.5623,154.311 68.3926,155.814 68.2299,157.317 
 68.0739,158.82 67.9248,160.323 67.7824,161.826 67.6467,163.329 67.5178,164.831 67.3955,166.334 67.2799,167.837 67.1709,169.34 67.0685,170.843 66.9727,172.346 
 66.8834,173.849 66.8007,175.352 66.7246,176.855 66.6549,178.357 66.5917,179.86 66.5351,181.363 66.4848,182.866 66.4411,184.368 66.4038,185.871 66.3729,187.374 
 66.3485,188.876 66.3305,190.379 66.3189,191.881 66.3138,193.384 66.3151,194.887 66.3227,196.389 66.3369,197.892 66.3574,199.394 66.3844,200.897 66.4178,202.4 
 66.4577,203.902 66.504,205.405 66.5567,206.908 66.616,208.411 66.6818,209.913 66.754,211.416 66.8328,212.919 66.9181,214.422 67.01,215.925 67.1083,217.428 
 67.2134,218.93 67.325,220.433 67.4432,221.936 67.5681,223.439 67.6997,224.942 67.8381,226.445 67.9831,227.948 68.135,229.451 68.2937,230.954 68.4592,232.457 
 68.6315,233.96 68.8108,235.462 68.997,236.965 69.1903,238.467 69.3905,239.97 69.5978,241.472 69.8122,242.974 70.0338,244.476 70.2626,245.977 70.4986,247.479 
 70.742,248.98 70.9927,250.481 71.2507,251.981 71.5163,253.481 71.7893,254.981 72.0699,256.48 72.358,257.978 72.6539,259.477 72.9575,260.974 73.2689,262.471 
 73.5882,263.967 73.9153,265.462 74.2505,266.956 74.5937,268.45 74.945,269.942 75.3044,271.433 75.6722,272.924 76.0482,274.413 76.4327,275.9 76.8256,277.387 
 77.227,278.872 77.6371,280.355 78.0559,281.837 78.4834,283.317 78.9198,284.795 79.3651,286.27 79.8194,287.744 80.2828,289.215 80.7554,290.684 81.2371,292.151 
 81.7283,293.615 82.2289,295.076 82.7391,296.534 83.2588,297.99 83.7883,299.441 84.3276,300.89 84.8767,302.335 85.4358,303.776 86.0049,305.213 86.5841,306.645 
 87.1736,308.073 87.7733,309.497 88.3835,310.915 89.0041,312.329 89.6352,313.737 90.2771,315.14 90.93,316.537 91.5937,317.929 92.2685,319.314 92.9544,320.692 
 93.6515,322.064 94.3598,323.429 95.0795,324.786 95.8106,326.135 96.5532,327.476 97.3073,328.809 98.0731,330.132 98.8506,331.447 99.6399,332.752 100.441,334.048 
 101.254,335.333 102.08,336.608 102.917,337.873 103.768,339.125 104.63,340.367 105.506,341.596 106.393,342.812 107.294,344.015 108.207,345.205 109.132,346.381 
 110.071,347.542 111.022,348.688 111.986,349.819 112.962,350.934 113.951,352.032 114.954,353.114 115.969,354.178 116.997,355.225 118.038,356.253 119.092,357.262 
 120.161,358.25 121.242,359.218 122.337,360.165 123.445,361.09 124.566,361.993 125.699,362.874 126.846,363.731 128.006,364.564 129.179,365.373 130.364,366.157 
 131.562,366.916 132.772,367.648 133.996,368.354 135.231,369.032 136.479,369.683 137.739,370.306 139.013,370.897 140.299,371.456 141.597,371.984 142.907,372.481 
 144.229,372.944 145.562,373.375 146.907,373.773 148.263,374.136 149.631,374.465 151.009,374.759 152.398,375.018 153.797

In [10]:
@btime(solve(problem));

  101.236 ms (41542 allocations: 5.34 MiB)


### Animation of the Figure 8

In [11]:
plot_xy_trailing(solution8, "./anim_eight_trailing.gif", ntrail=3, duration = 7.0 )

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_eight_trailing.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_eight_trailing.gif")

In [12]:
plot_xy_scattering(solution8,"./anim_eight_scattering.gif", 6.0)

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_eight_scattering.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_eight_scattering.gif")

### The Moth I

In [13]:
G = 1
m1 = MassBody(1.0, SVector(-1.0, 0.0, 0.0), SVector(0.46444, 0.39606, 0.0))
m2 = MassBody(1.0, SVector(1.0, 0.0, 0.0), SVector(0.46444, 0.39606, 0.0))
m3 = MassBody(1.0, SVector(0.0, 0.0, 0.0), SVector(-2*0.46444, -2*0.39606, 0.0))
tspan = (0.0, 15.0);
problem = NBodyGravProblem([m1, m2, m3], tspan, G)
solution_moth = solve(problem);
pl =plot(solution_moth, vars = (1,2))
plot!(pl,solution_moth, vars = (4,5))
plot!(pl,solution_moth, vars = (7,8),xlim=(-1.1,1.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip0302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 64.0844,193.804 64.6266,193.506 65.1756,193.208 65.7317,192.91 66.2948,192.612 66.865,192.314 67.4425,192.017 68.0272,191.719 68.6194,191.422 69.219,191.125 
 69.8261,190.828 70.4409,190.531 71.0634,190.235 71.6937,189.939 72.3319,189.643 72.9781,189.348 73.6323,189.053 74.2947,188.758 74.9654,188.464 75.6445,188.171 
 76.3321,187.878 77.0282,187.585 77.733,187.294 78.4466,187.003 79.169,186.712 79.9005,186.423 80.641,186.134 81.3908,185.847 82.15,185.56 82.9186,185.274 
 83.6968,184.989 84.4846,184.706 85.2824,184.423 86.09,184.142 86.9077,183.863 87.7356,183.584 88.5739,183.308 89.4226,183.032 90.282,182.759 91.1521,182.487 
 92.0331,182.217 92.9252,181.949 93.8284,181.683 94.7429,181.419 95.6689,181.157 96.6065,180.898 97.5559,180.641 98.5171,180.387 99.4905,180.136 100.476,179.887 
 101.474,179.641 102.485,179.399 103.508,179.159 104.545,178.924 105.594,178.691 106.657,178.463 107.733,178.239 108.822,178.019 109.925,177.803 111.042,177.592 
 112.173,177.386 113.318,177.184 114.478,176.988 115.652,176.797 116.841,176.611 118.045,176.432 119.264,176.259 120.498,176.093 121.747,175.934 123.011,175.781 
 124.29,175.637 125.585,175.5 126.896,175.371 128.223,175.251 129.566,175.139 130.926,175.037 132.302,174.945 133.693,174.863 135.1,174.791 136.524,174.731 
 137.962,174.682 139.416,174.645 140.886,174.62 142.371,174.608 143.872,174.61 145.389,174.626 146.922,174.657 148.469,174.704 150.031,174.766 151.607,174.845 
 153.196,174.941 154.798,175.054 156.412,175.184 158.038,175.334 159.675,175.502 161.323,175.692 162.981,175.902 164.648,176.134 166.323,176.387 168.004,176.662 
 169.692,176.96 171.383,177.281 173.078,177.626 174.775,177.995 176.472,178.389 178.169,178.807 179.863,179.249 181.554,179.716 183.239,180.209 184.917,180.726 
 186.586,181.269 188.245,181.837 189.892,182.428 191.527,183.044 193.147,183.684 194.753,184.348 196.339,185.034 197.906,185.742 199.453,186.472 200.978,187.223 
 202.481,187.994 203.961,188.784 205.418,189.594 206.85,190.421 208.255,191.265 209.634,192.125 210.986,192.999 212.31,193.887 213.608,194.788 214.878,195.701 
 216.119,196.627 217.333,197.562 218.52,198.507 219.679,199.46 220.811,200.421 221.916,201.389 222.993,202.364 224.045,203.344 225.069,204.33 226.069,205.319 
 227.043,206.312 227.993,207.309 228.919,208.307 229.822,209.308 230.701,210.31 231.557,211.314 232.391,212.317 233.204,213.321 233.996,214.325 234.769,215.329 
 235.521,216.331 236.254,217.333 236.969,218.333 237.665,219.331 238.344,220.327 239.005,221.321 239.65,222.313 240.279,223.302 240.893,224.289 241.492,225.272 
 242.076,226.253 242.645,227.23 243.201,228.204 243.744,229.175 244.274,230.142 244.791,231.105 245.296,232.065 245.79,233.02 246.272,233.973 246.743,234.921 
 247.204,235.865 247.654,236.805 248.094,237.741 248.525,238.673 248.946,239.601 249.359,240.524 249.762,241.444 250.157,242.359 250.544,243.27 250.923,244.177 
 251.294,245.079 251.657,245.978 252.014,246.872 252.363,247.761 252.706,248.647 253.042,249.528 253.371,250.404 253.695,251.276 254.012,252.144 254.324,253.008 
 254.63,253.868 254.931,254.723 255.226,255.574 255.516,256.421 255.802,257.264 256.082,258.102 256.358,258.936 256.629,259.766 256.896,260.592 257.159,261.413 
 257.418,262.231 257.673,263.044 257.923,263.853 258.17,264.658 258.414,265.459 258.654,266.256 258.89,267.049 259.124,267.838 259.354,268.623 259.58,269.403 
 259.804,270.18 260.025,270.953 260.244,271.722 260.459,272.487 260.672,273.248 260.882,274.005 261.09,274.758 261.295,275.508 261.498,276.253 261.699,276.995 
 261.897,277.733 262.094,278.467 262.288,279.198 262.48,279.924 262.671,280.647 262.859,281.366 263.046,282.082 263.231,282.794 263.415,

### Animation of the Moth I

In [21]:
plot_xy_trailing(solution_moth, "./anim_moth_trailing.gif", ntrail = 4, duration = 15.0)

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_moth_trailing.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_moth_trailing.gif")

In [28]:
plot_xy_scattering(solution_moth,"./anim_moth_scattering.gif", 15.0)

INFO: Saved animation to /mnt/juliabox/NBodyProblem/anim_moth_scattering.gif


Plots.AnimatedGif("/mnt/juliabox/NBodyProblem/anim_moth_scattering.gif")

## Using DiffPhysics.jl

In [23]:
using DiffEqPhysics

### The Figure 8

In [24]:
G = 1.0
M = [1.0, 1.0, 1.0]
invM = inv.(M)

pos_x = [b1.r[1], b2.r[1], b3.r[1]]
pos_y = [b1.r[2], b2.r[2], b3.r[2]]
pos_z = [b1.r[3], b2.r[3], b3.r[3]]
pos = ArrayPartition(pos_x,pos_y,pos_z)

vel_x = [b1.v[1], b2.v[1], b3.v[1]]
vel_y = [b1.v[2], b2.v[2], b3.v[2]]
vel_z = [b1.v[3], b2.v[3], b3.v[3]]
vel = ArrayPartition(vel_x,vel_y,vel_z)

tspan = (0.0, 2.0)

const ∑ = sum
const N = 3
potential(p, t, x, y, z, M) = -G*∑(i->∑(j->(M[i]*M[j])/sqrt((x[i]-x[j])^2 + (y[i]-y[j])^2 + (z[i]-z[j])^2), 1:i-1), 2:N)
nprob = NBodyProblem(potential, M, vel, pos, tspan)
sol = solve(nprob,Yoshida6(), dt=0.01)
pl = plot(sol, vars = (10,13))
plot!(pl,sol, vars = (11,14))
plot!(pl,sol, vars = (12,15),xlim=(-1.1,1.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -0.3 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
<polyline clip-path="url(#clip2902)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 71.4581,70.2536 71.3441,70.4725 71.2305,70.6915 71.1171,70.9108 71.0041,71.1301 70.8913,71.3496 70.7788,71.5693 70.6667,71.7891 70.5548,72.009 70.4432,72.2291 
 70.3319,72.4494 70.2209,72.6698 70.1101,72.8903 69.9997,73.111 69.8896,73.3318 69.7797,73.5527 69.6702,73.7738 69.5609,73.995 69.4519,74.2164 69.3432,74.4379 
 69.2348,74.6595 69.1267,74.8812 69.0189,75.1031 68.9113,75.3252 68.8041,75.5473 68.6971,75.7696 68.5904,75.992 68.484,76.2146 68.3779,76.4372 68.272,76.66 
 68.1665,76.8829 68.0612,77.106 67.9562,77.3291 67.8516,77.5524 67.7471,77.7758 67.643,77.9994 67.5391,78.223 67.4356,78.4468 67.3323,78.6707 67.2293,78.8947 
 67.1265,79.1188 67.0241,79.3431 66.9219,79.5674 66.82,79.7919 66.7184,80.0165 66.6171,80.2412 66.516,80.466 66.4152,80.6909 66.3147,80.9159 66.2145,81.1411 
 66.1145,81.3663 66.0149,81.5917 65.9155,81.8171 65.8163,82.0427 65.7175,82.2683 65.6189,82.4941 65.5206,82.72 65.4226,82.946 65.3248,83.1721 65.2273,83.3982 
 65.1301,83.6245 65.0332,83.8509 64.9365,84.0774 64.8401,84.304 64.7439,84.5307 64.6481,84.7574 64.5525,84.9843 64.4572,85.2113 64.3621,85.4383 64.2673,85.6655 
 64.1728,85.8927 64.0786,86.1201 63.9846,86.3475 63.8909,86.575 63.7974,86.8027 63.7043,87.0304 63.6114,87.2582 63.5187,87.486 63.4263,87.714 63.3342,87.9421 
 63.2424,88.1702 63.1508,88.3984 63.0594,88.6268 62.9684,88.8552 62.8776,89.0836 62.787,89.3122 62.6968,89.5408 62.6068,89.7696 62.517,89.9984 62.4275,90.2273 
 62.3383,90.4562 62.2493,90.6853 62.1606,90.9144 62.0722,91.1436 61.984,91.3729 61.896,91.6022 61.8084,91.8317 61.7209,92.0612 61.6338,92.2908 61.5469,92.5204 
 61.4602,92.7502 61.3739,92.98 61.2877,93.2098 61.2019,93.4398 61.1162,93.6698 61.0309,93.8999 60.9458,94.13 60.8609,94.3603 60.7763,94.5906 60.692,94.8209 
 60.6079,95.0514 60.524,95.2819 60.4404,95.5124 60.3571,95.7431 60.274,95.9738 60.1912,96.2045 60.1086,96.4353 60.0263,96.6662 59.9442,96.8972 59.8624,97.1282 
 59.7808,97.3593 59.6994,97.5904 59.6184,97.8216 59.5375,98.0529 59.4569,98.2842 59.3766,98.5156 59.2965,98.747 59.2167,98.9785 59.1371,99.2101 59.0577,99.4417 
 58.9786,99.6733 58.8998,99.9051 58.8211,100.137 58.7428,100.369 58.6647,100.601 58.5868,100.833 58.5091,101.065 58.4318,101.297 58.3546,101.529 58.2777,101.761 
 58.201,101.993 58.1246,102.225 58.0484,102.458 57.9725,102.69 57.8968,102.922 57.8214,103.155 57.7461,103.387 57.6712,103.62 57.5964,103.853 57.5219,104.085 
 57.4477,104.318 57.3737,104.551 57.2999,104.783 57.2264,105.016 57.1531,105.249 57.08,105.482 57.0072,105.715 56.9346,105.948 56.8622,106.181 56.7901,106.414 
 56.7182,106.647 56.6466,106.881 56.5752,107.114 56.504,107.347 56.4331,107.581 56.3623,107.814 56.2919,108.047 56.2216,108.281 56.1516,108.514 56.0819,108.748 
 56.0123,108.981 55.943,109.215 55.8739,109.449 55.8051,109.682 55.7365,109.916 55.6681,110.15 55.5999,110.383 55.532,110.617 55.4643,110.851 55.3969,111.085 
 55.3296,111.319 55.2626,111.553 55.1959,111.787 55.1293,112.021 55.063,112.255 54.9969,112.489 54.931,112.723 54.8654,112.958 54.8,113.192 54.7348,113.426 
 54.6699,113.66 54.6051,113.895 54.5406,114.129 54.4764,114.363 54.4123,114.598 54.3485,114.832 54.2849,115.067 54.2215,115.301 54.1583,115.536 54.0954,115.77 
 54.0327,116.005 53.9702,116.239 53.908,116.474 53.8459,116.709 53.7841,116.943 53.7225,117.178 53.6611,117.413 53.6,117.647 53.539,117.882 53.4783,118.117 
 53.4178,118.352 53.3576,118.587 53.2975,118.822 53.2377,119.057 53.1781,119.292 53.1187,119.527 53.0595,119.762 53.0005,119.997 52.9418,120.232 52.8833,120.467 
 52.825,120.702 52.7669,120.937 52.709,121.172 52.6514,121.407 52.5939,121.642 52.5367,121.877 52.4797,122.113 52.4229,122.34

In [25]:
@btime( solve(nprob));

  4.753 ms (126977 allocations: 2.40 MiB)


### The Moth I

In [26]:
G = 1.0
M = [1.0, 1.0, 1.0]
invM = inv.(M)

pos_x = [m1.r[1], m2.r[1], m3.r[1]]
pos_y = [m1.r[2], m2.r[2], m3.r[2]]
pos_z = [m1.r[3], m2.r[3], m3.r[3]]
pos = ArrayPartition(pos_x,pos_y,pos_z)

vel_x = [m1.v[1], m2.v[1], m3.v[1]]
vel_y = [m1.v[2], m2.v[2], m3.v[2]]
vel_z = [m1.v[3], m2.v[3], m3.v[3]]
vel = ArrayPartition(vel_x,vel_y,vel_z)

tspan = (0.0, 15.0)

const ∑ = sum
const N = 3
potential(p, t, x, y, z, M) = -G*∑(i->∑(j->(M[i]*M[j])/sqrt((x[i]-x[j])^2 + (y[i]-y[j])^2 + (z[i]-z[j])^2), 1:i-1), 2:N)
nprob = NBodyProblem(potential, M, vel, pos, tspan)
sol = solve(nprob,Yoshida6(), dt=0.01)
pl = plot(sol, vars = (10,13))
plot!(pl,sol, vars = (11,14))
plot!(pl,sol, vars = (12,15),xlim=(-1.1,1.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip3102)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 64.0844,193.833 64.1986,193.769 64.3131,193.706 64.4279,193.643 64.543,193.579 64.6584,193.516 64.7741,193.453 64.8902,193.389 65.0065,193.326 65.1232,193.263 
 65.2401,193.2 65.3574,193.136 65.475,193.073 65.5929,193.01 65.7111,192.946 65.8297,192.883 65.9485,192.82 66.0677,192.756 66.1872,192.693 66.307,192.63 
 66.4272,192.567 66.5476,192.503 66.6684,192.44 66.7895,192.377 66.9109,192.314 67.0327,192.25 67.1547,192.187 67.2771,192.124 67.3999,192.061 67.5229,191.997 
 67.6463,191.934 67.77,191.871 67.894,191.808 68.0184,191.744 68.1431,191.681 68.2681,191.618 68.3935,191.555 68.5192,191.492 68.6452,191.429 68.7715,191.365 
 68.8982,191.302 69.0253,191.239 69.1526,191.176 69.2803,191.113 69.4084,191.05 69.5368,190.987 69.6655,190.924 69.7945,190.86 69.9239,190.797 70.0537,190.734 
 70.1838,190.671 70.3142,190.608 70.445,190.545 70.5761,190.482 70.7076,190.419 70.8394,190.356 70.9716,190.293 71.1041,190.23 71.2369,190.167 71.3702,190.104 
 71.5037,190.042 71.6376,189.979 71.7719,189.916 71.9065,189.853 72.0415,189.79 72.1769,189.727 72.3126,189.664 72.4486,189.602 72.585,189.539 72.7218,189.476 
 72.8589,189.413 72.9964,189.35 73.1343,189.288 73.2725,189.225 73.4111,189.162 73.55,189.1 73.6893,189.037 73.829,188.974 73.969,188.912 74.1095,188.849 
 74.2502,188.787 74.3914,188.724 74.5329,188.662 74.6748,188.599 74.8171,188.537 74.9597,188.474 75.1028,188.412 75.2462,188.349 75.3899,188.287 75.5341,188.225 
 75.6786,188.162 75.8235,188.1 75.9688,188.038 76.1145,187.976 76.2605,187.913 76.407,187.851 76.5538,187.789 76.701,187.727 76.8486,187.665 76.9966,187.603 
 77.1449,187.541 77.2937,187.479 77.4428,187.417 77.5924,187.355 77.7423,187.293 77.8926,187.231 78.0434,187.169 78.1945,187.107 78.346,187.045 78.4979,186.983 
 78.6502,186.922 78.8029,186.86 78.956,186.798 79.1096,186.737 79.2635,186.675 79.4178,186.613 79.5725,186.552 79.7276,186.49 79.8832,186.429 80.0391,186.368 
 80.1955,186.306 80.3522,186.245 80.5094,186.184 80.667,186.122 80.825,186.061 80.9834,186 81.1422,185.939 81.3015,185.878 81.4612,185.817 81.6212,185.756 
 81.7818,185.695 81.9427,185.634 82.104,185.573 82.2658,185.512 82.428,185.451 82.5906,185.39 82.7537,185.33 82.9171,185.269 83.081,185.209 83.2454,185.148 
 83.4102,185.088 83.5754,185.027 83.741,184.967 83.9071,184.906 84.0736,184.846 84.2405,184.786 84.4079,184.726 84.5757,184.665 84.744,184.605 84.9127,184.545 
 85.0818,184.485 85.2514,184.425 85.4214,184.366 85.5919,184.306 85.7628,184.246 85.9342,184.186 86.106,184.127 86.2783,184.067 86.4511,184.008 86.6242,183.948 
 86.7979,183.889 86.972,183.83 87.1465,183.77 87.3216,183.711 87.497,183.652 87.673,183.593 87.8494,183.534 88.0262,183.475 88.2036,183.416 88.3814,183.357 
 88.5596,183.299 88.7384,183.24 88.9176,183.182 89.0972,183.123 89.2774,183.065 89.458,183.006 89.6391,182.948 89.8207,182.89 90.0027,182.832 90.1853,182.774 
 90.3683,182.716 90.5518,182.658 90.7358,182.6 90.9202,182.542 91.1052,182.485 91.2906,182.427 91.4765,182.37 91.6629,182.312 91.8499,182.255 92.0373,182.198 
 92.2251,182.141 92.4135,182.084 92.6024,182.027 92.7918,181.97 92.9817,181.913 93.1721,181.856 93.363,181.8 93.5544,181.743 93.7463,181.687 93.9387,181.631 
 94.1316,181.574 94.325,181.518 94.5189,181.462 94.7134,181.406 94.9083,181.35 95.1038,181.295 95.2998,181.239 95.4963,181.184 95.6933,181.128 95.8909,181.073 
 96.089,181.018 96.2876,180.963 96.4867,180.908 96.6863,180.853 96.8865,180.798 97.0872,180.744 97.2884,180.689 97.4902,180.635 97.6925,180.58 97.8954,180.526 
 98.0987,180.472 98.3026,180.418 98.5071,180.364 98.7121,180.311 98.9176,180.257 99.1237,180.204 99.3303,180.15 99.5375,180.097 99.7452,180.044 99.9535,179.991 
 100.

## Benchmarks

In [27]:
using BenchmarkTools

for n = [18,19,20,21,22,23,24,25, 30, 35, 40, 45, 50]
    bodies = [MassBody(1.0, SVector{3}(100*rand(3)), SVector{3}(100*rand(3))) for i =1:n];
    tspan=(0.0, 10.0);
    problem = NBodyGravProblem(bodies, tspan, 1)
    #@btime(solve(problem))
end